<a href="https://colab.research.google.com/github/ayanand/CapstoneML/blob/main/polynomial_kernel_SVM_analysiscap3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ayanand/CapstoneML/blob/main/polynomial_kernel_SVM_analysiscap3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [ ]:
import pandas as pd
!pip install fsspec
!pip install gcsfs 
 
 


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import confusion_matrix
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv('gs://capstoragebucket2020/All_joined_v5.csv')
df=df.sort_values(by=['year', 'quarter','latitude','longitude'])
df.tail()

,latitude,longitude,quarter,year,incendie_count,alarmes_incendies_count,total_crimes,vols_count,mefait_count,vol_de_vehicule_count,introduction_count,vol_moteur_count,infractions_entrainant_count,sum_etage_hors_sol,sum_nombre_logement,min_annee_construction,max_annee_construction,avg_annee_construction,sum_superficie_terrain,area,population,dwellings,households,averagehouseholdsize,averageage,averagesizeofcensusfamilies,workers,caserne_count,last_date,incendie_count_last_100,incendie_count_last_300,alarm_incendie_count_last_100
11709,45.67,-73.50,4,2020,0,0,2,0,0,0,1,1,0,1706,2237,1914,2019,1969.399127,518847,0.11503,505,285,273,1.8,43.2,2.6,100,0,2020-12-01,0,1,0
11738,45.68,-73.50,4,2020,0,0,5,0,1,3,0,1,0,1377,898,1959,2019,1994.756098,465540,0.45534,821,422,386,2.1,37.5,2.7,335,0,2020-12-01,0,2,0
11760,45.69,-73.50,4,2020,0,0,2,0,0,0,2,0,0,254,285,1959,2018,1999.281967,75473,2.99229,1350,519,518,2.6,38.5,2.9,580,0,2020-12-01,1,2,0
11784,45.69,-73.49,4,2020,0,0,3,0,0,1,0,2,0,1754,1687,1925,2017,1990.051636,493065,0.20290,677,290,286,2.4,40.4,2.7,195,0,2020-12-01,0,0,1
11819,45.70,-73.49,4,2020,0,0,1,0,0,0,0,1,0,933,805,1900,2019,1997.866756,177975,0.48567,645,255,249,2.6,41.3,2.9,170,1,2020-12-01,0,4,1


In [ ]:
print(df['incendie_count'].value_counts())

#Define three classes of fire risk by the summed number of fire incidence from 2015-2020
#low risk < 5, moderate risk 5-15, high risk > 15
df['fire'] = df['incendie_count']
df['fire'][df['incendie_count']==0] = 0
df['fire'][(df['incendie_count']>=1) & (df['incendie_count']<4)] = 1
df['fire'][df['incendie_count']>=4] = 2


df_new = df
df_new['year'] =2021- df_new['year']
  

0     7633
1     2644
2      961
3      362
4      151
5       55
6       19
7        9
9        4
8        4
10       1
Name: incendie_count, dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import geopy.distance

def dist(x,y):
  coords_1 = (45.504654, -73.56546)
  coords_2 = (x, y)
  return geopy.distance.vincenty(coords_1, coords_2).km

df_new['distance']=0
for x in range(0,len(df_new)-1):
  df_new['distance'][x]=dist(df_new['latitude'][x], df_new['longitude'][x])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
df_new.tail(10)

,latitude,longitude,quarter,year,incendie_count,alarmes_incendies_count,total_crimes,vols_count,mefait_count,vol_de_vehicule_count,introduction_count,vol_moteur_count,infractions_entrainant_count,sum_etage_hors_sol,sum_nombre_logement,min_annee_construction,max_annee_construction,avg_annee_construction,sum_superficie_terrain,area,population,dwellings,households,averagehouseholdsize,averageage,averagesizeofcensusfamilies,workers,caserne_count,last_date,incendie_count_last_100,incendie_count_last_300,alarm_incendie_count_last_100,fire,distance
11545,45.66,-73.51,4,1,0,0,5,0,0,4,0,1,0,1207,1044,1941,2017,1984.169195,494180,0.28223,448,168,167,2.6,41.7,3.0,135,0,2020-12-01,0,0,2,0,17
11569,45.66,-73.50,4,1,0,0,2,0,1,1,0,0,0,1953,1624,1925,2014,1976.802276,557374,0.30364,1090,395,388,2.8,39.9,3.0,290,0,2020-12-01,0,1,2,0,18
11593,45.66,-73.49,4,1,0,0,1,0,0,1,0,0,0,490,542,1752,2018,1976.853933,178758,0.21275,487,212,210,2.2,47.9,2.6,110,0,2020-12-01,0,0,1,0,18
11617,45.67,-73.55,4,1,0,0,1,0,0,1,0,0,0,874,607,1900,2017,1994.664671,196023,0.72304,453,158,154,2.7,43.1,2.7,75,0,2020-12-01,0,0,1,0,18
11685,45.67,-73.51,4,1,0,2,2,0,0,0,0,2,0,459,576,1920,2015,1981.962264,338851,0.73267,720,345,343,2.0,47.3,2.7,165,1,2020-12-01,0,0,4,0,18
11709,45.67,-73.50,4,1,0,0,2,0,0,0,1,1,0,1706,2237,1914,2019,1969.399127,518847,0.11503,505,285,273,1.8,43.2,2.6,100,0,2020-12-01,0,1,0,0,19
11738,45.68,-73.50,4,1,0,0,5,0,1,3,0,1,0,1377,898,1959,2019,1994.756098,465540,0.45534,821,422,386,2.1,37.5,2.7,335,0,2020-12-01,0,2,0,0,20
11760,45.69,-73.50,4,1,0,0,2,0,0,0,2,0,0,254,285,1959,2018,1999.281967,75473,2.99229,1350,519,518,2.6,38.5,2.9,580,0,2020-12-01,1,2,0,0,21
11784,45.69,-73.49,4,1,0,0,3,0,0,1,0,2,0,1754,1687,1925,2017,1990.051636,493065,0.20290,677,290,286,2.4,40.4,2.7,195,0,2020-12-01,0,0,1,0,21
11819,45.70,-73.49,4,1,0,0,1,0,0,0,0,1,0,933,805,1900,2019,1997.866756,177975,0.48567,645,255,249,2.6,41.3,2.9,170,1,2020-12-01,0,4,1,0,22


In [ ]:
df_new = df_new.drop("incendie_count", axis=1)
df_new = df_new.drop("last_date", axis=1)
dv_savepoint = df_new

In [ ]:
df_new = df_new.drop("latitude", axis=1)
df_new = df_new.drop("longitude", axis=1)

df_new = df_new.drop("min_annee_construction", axis=1)
df_new = df_new.drop("max_annee_construction", axis=1)

df_new = df_new.drop("incendie_count_last_300", axis=1)
df_new = df_new.drop("incendie_count_last_100", axis=1)
df_new = df_new.drop("alarm_incendie_count_last_100", axis=1)

In [ ]:
y= df_new["fire"].copy()
X=df_new.drop("fire", axis=1)

In [ ]:
len(X),len(y)

(11843, 11843)

In [ ]:
corr_matrix = df_new.corr()
corr_matrix["fire"].sort_values(ascending=False)

fire                            1.000000
sum_nombre_logement             0.479032
total_crimes                    0.429216
sum_etage_hors_sol              0.420503
alarmes_incendies_count         0.410367
mefait_count                    0.399054
introduction_count              0.395315
vols_count                      0.312964
vol_moteur_count                0.309208
vol_de_vehicule_count           0.209143
caserne_count                   0.158662
max_annee_construction          0.119309
sum_superficie_terrain          0.084048
dwellings                       0.061831
infractions_entrainant_count    0.052661
households                      0.036779
year                            0.014774
quarter                        -0.031297
averageage                     -0.038386
population                     -0.051500
averagesizeofcensusfamilies    -0.071044
workers                        -0.083336
averagehouseholdsize           -0.167736
min_annee_construction         -0.172250
area            

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

In [ ]:
X=pd.DataFrame(X)
X

In [ ]:
# manually prepare test and train data 

X_train=X[df_new['year'] > 2] # choose 2019 data as train
y_train = y[df_new['year'] > 2]

X_test = X[df_new['year']*df_new['quarter'] == 2]
y_test = y[df_new['year']*df_new['quarter'] == 2]

len(X_train),len(X_test),len(y_train),len(y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(7999, 985, 7999, 985)

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

#X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1,random_state=4)

len(X_train),len(X_test),len(y_train),len(y_test)

(7999, 985, 7999, 985)

In [ ]:
y_train.value_counts(), y_test.value_counts()

(0    5111
 1    2730
 2     158
 Name: fire, dtype: int64, 0    594
 1    359
 2     32
 Name: fire, dtype: int64)

In [ ]:
## SVM with polynomial kernel 


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import LinearSVC

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)),
        ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42, max_iter=3000))
    ])

polynomial_svm_clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=3000, multi_class='ovr',
                           penalty='l2', random_state=42, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [ ]:
y_predictions=polynomial_svm_clf.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.8029134765100293
[[364 203  27]
 [225 116  18]
 [ 17  13   2]]


In [ ]:
## polynomial kernel ; kernel trick ??


from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
        ("svm_clf", SVC(kernel="poly", degree=5, coef0=10, C=5))
    ])
## poly_kernel_svm_clf.fit(X_train, y_train)## taking lot of time

In [ ]:

from sklearn.tree import DecisionTreeClassifier



tree_clf = DecisionTreeClassifier(max_depth=100, random_state=42)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=100, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [ ]:
y_predictions=tree_clf.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.7984757051887363
[[355 224  15]
 [233 113  13]
 [ 22  10   0]]
